In [1]:
from kafka.admin import KafkaAdminClient
from kafka import KafkaProducer
from kafka.admin import NewTopic
import kafka
import os
import boto3
import pandas as pd
from tqdm import tqdm
import json
import time

In [2]:
KAFKA_HOME = '/root/kafka_2.13-3.2.1'
KAFKA_BOOTSTRAP_SERVERS = ['localhost:9092',]

In [3]:
# producer definition from IP address given before
producer = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS)

# connect to the cluster to run admin functions
kafka_admin = KafkaAdminClient(
        bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
        api_version=(0,10,2))

"""
#topic

#kafka_admin.delete_topics(['stream'])

stream = NewTopic(name='stream', 
                       num_partitions=2, 
                       replication_factor=1)

kafka_admin.create_topics(new_topics=[stream])

kafka_admin.list_topics()"""

"\n#topic\n\n#kafka_admin.delete_topics(['stream'])\n\nstream = NewTopic(name='stream', \n                       num_partitions=2, \n                       replication_factor=1)\n\nkafka_admin.create_topics(new_topics=[stream])\n\nkafka_admin.list_topics()"

In [4]:
s3_client = boto3.client('s3',
                         endpoint_url='https://cloud-areapd.pd.infn.it:5210',
                         aws_access_key_id='1a4543841b844a88bb3f2eba45764d61',
                         aws_secret_access_key='42e2f16592f54668b8421ecf5ca7ba51',
                         verify=False)

In [5]:
list_obj_contents = s3_client.list_objects(Bucket= 'mapd-minidt-stream')['Contents']
list_obj_contents[0:5]

/root/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloud-areapd.pd.infn.it'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[{'Key': 'data_000000.txt',
  'LastModified': datetime.datetime(2022, 5, 30, 8, 56, 44, 485000, tzinfo=tzlocal()),
  'ETag': '"0a7dccb174b6732af3c9e040109df403-2"',
  'Size': 29604978,
  'StorageClass': 'STANDARD',
  'Owner': {'DisplayName': 'PhysicsOfData-students',
   'ID': 'd2e941ce4b324467b6b3d467a923a9bc$d2e941ce4b324467b6b3d467a923a9bc'}},
 {'Key': 'data_000001.txt',
  'LastModified': datetime.datetime(2022, 5, 30, 8, 56, 44, 580000, tzinfo=tzlocal()),
  'ETag': '"777544f2f3b53cc8043791bde33c375a-2"',
  'Size': 29757766,
  'StorageClass': 'STANDARD',
  'Owner': {'DisplayName': 'PhysicsOfData-students',
   'ID': 'd2e941ce4b324467b6b3d467a923a9bc$d2e941ce4b324467b6b3d467a923a9bc'}},
 {'Key': 'data_000002.txt',
  'LastModified': datetime.datetime(2022, 5, 30, 8, 56, 44, 644000, tzinfo=tzlocal()),
  'ETag': '"d7f465cb61aafab424073b58f4cc1aa1-2"',
  'Size': 30921744,
  'StorageClass': 'STANDARD',
  'Owner': {'DisplayName': 'PhysicsOfData-students',
   'ID': 'd2e941ce4b324467b6b3d467a9

In [6]:
n = len(list_obj_contents)
for i in range(0,n):
    file_name = list_obj_contents[i]['Key']
    row_data = pd.read_csv(s3_client.get_object(Bucket='mapd-minidt-stream', 
                                                Key=file_name).get('Body'), dtype='str')
        
    # For loop over file size
    for j in tqdm(range(0, len(row_data))):
        # dictionaries creation from dataframe's rows
        jj = row_data.iloc[j].to_dict()
        # unnecessary floats are cast into ints
        for key in ['HEAD', 'FPGA', "TDC_CHANNEL"]:
            jj[key] = int(jj[key])
        # json row is sent to the Kafka topic 'topic_stream'
        producer.send('stream', json.dumps(jj).encode('utf-8'))
        time.sleep(0.0003)
    producer.flush()


/root/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloud-areapd.pd.infn.it'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
100%|██████████████████████████████████████████████| 1310592/1310592 [26:16<00:00, 831.16it/s]
/root/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloud-areapd.pd.infn.it'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
100%|██████████████████████████████████████████████| 1310720/1310720 [26:21<00:00, 829.02it/s]
/root/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cloud-areapd.pd.

KeyboardInterrupt: 